In [79]:
import requests
import json

In [109]:
import time

url = "http://localhost:8000/query"
data = {
    "question": "Who was Carl Friedrich Gauss?",
    "subject": "math",
}
lines = []
start = time.time()
with requests.post(url, json=data, stream=True) as r:
    r.raise_for_status()
    mode = None
    for line in r.iter_lines():
        lines.append(line)
        line_dict = json.loads(line)
        if len(lines) == 1:
            thread_id = line_dict["thread_id"]
            print(f"{thread_id=}")
        model_repr = line_dict["model_repr"]
        tag = line_dict["tag"]
        response_type = line_dict["response_type"]
        response = line_dict["response"]
        if mode != (model_repr, tag, response_type):
            mode = (model_repr, tag, response_type)
            print()
            print("="*100)
            print(f"{mode=}")
            print("="*100)
        print(response, end="")
end = time.time()
time_taken = end - start
print()
print(f"Total Time taken: {time_taken}")

thread_id='181ec624-cbe9-46a3-93c7-b952aafe5230'

mode=("ChatOllama(model='qwen3:4b', reasoning=True, temperature=0.0, base_url='http://localhost:11434/')", 'qa', 'reasoning')
We are given a question: "Who was Carl Friedrich Gauss?"

We have several chunks of context. We need to look for information about Carl Friedrich Gauss in the context.

Let's go through the chunks:

Chunk ID: 33ce5048-3a48-07e1-e86b-3e4fdbebbe7b
This chunk contains information about the Fundamental Theorem of Arithmetic and mentions Carl Friedrich Gauss.

Specifically, in this chunk we have:
"An equivalent version version of Theorem 1.2 was probably first recorded as Proposition 14 of Book IX in Euclid's Elements, before it came to be known as the Fundamental Theorem of Arithmetic. However, the first correct proof was given by Carl Friedrich Gauss in his Disquisitiones Arithmeticae."

Also: "Carl Friedrich Gauss is often referred to as the 'Prince of Mathematicians' and is considered one of the three greatest mat